In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/song_lyrics_en.csv')

df.head()

,Unnamed: 0,tag,artist,views,popularity_bin,song_document
0,0,rap,Cam'ron,173166,Medium,"Killa Cam Killa Cam, Killa Cam, Cam\nKilla Cam..."
1,1,rap,JAY-Z,468624,Medium,"Can I Live Yeah, hah, yeah, Roc-A-Fella\nWe in..."
2,2,rap,Fabolous,4743,Very Low,Forgive Me Father Maybe cause I'm eatin\nAnd t...
3,3,rap,Cam'ron,144404,Medium,"Down and Out Ugh, Killa!\nBaby!\nKanye, this t..."
4,4,rap,Lil Wayne,78271,Low,"Fly In So they ask me\n""Young boy\nWhat you go..."


In [4]:
slice_num = 10000
df = df[:slice_num]

In [6]:
! python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 53.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_pandas

tqdm_pandas(tqdm())
spacy.prefer_gpu()

nlp = spacy.load('en_core_web_md')
tag_embeddings = np.load('/content/drive/MyDrive/Colab Notebooks/tag_embeddings.npy')
artist_embeddings = np.load('/content/drive/MyDrive/Colab Notebooks/artist_embeddings.npy')

tag_embeddings


0it [00:00, ?it/s]
0it [00:34, ?it/s]


array([[[-0.04503907,  0.0417758 ,  0.01997494]],

       [[-0.04503907,  0.0417758 ,  0.01997494]],

       [[-0.04503907,  0.0417758 ,  0.01997494]],

       ...,

       [[-0.03446241,  0.01993293,  0.01982058]],

       [[-0.03446241,  0.01993293,  0.01982058]],

       [[ 0.00429112,  0.00109095,  0.01435422]]], dtype=float32)

In [11]:
artist_embeddings.shape

(10000, 600)

In [13]:
tag_embeddings.shape

(10000, 3)

In [16]:
def transform_y(label):
    if label == 'Very Low':
        return 0
    if label == 'Low':
        return 1
    if label == 'Medium':
        return 2
    if label == 'High':
        return 3
    if label == 'Very High':
        return 4

In [25]:
import cupy as cp  # 確保安裝了 cupy
print("Num GPUs Available: ", cp.cuda.runtime.getDeviceCount())

Num GPUs Available:  1


In [26]:
import spacy
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_pandas


def get_word_vector(text):
    doc = nlp(text)
    vectors = [token.vector for token in doc if token.has_vector]

    if len(vectors) > 0:
        return cp.mean(cp.array(vectors), axis=0)  # 使用 cupy 的平均計算
    else:
        return cp.zeros((300,))  # 返回零向量

# 假設 df 是已經存在的 DataFrame
# 使用進度條處理文本
df['vector'] = df['song_document'].progress_apply(get_word_vector)

# 將詞向量轉換為 NumPy 陣列，檢查形狀
X = cp.vstack(df['vector'].values)
print(X.shape)  # 確認 X 的形狀

100%|██████████| 10000/10000 [12:46<00:00, 13.05it/s]


(10000, 300)


In [ ]:
np.save('data/song_document_w2v.npy', X)

In [ ]:
import numpy as np

np.load('data/song_document_w2v.npy').shape

(10000, 300)

In [37]:
X = cp.hstack([X, tag_embeddings, artist_embeddings])


In [46]:
labels = df['popularity_bin'].values
# 使用 numpy 的向量化方式進行轉換
transformed_labels_np = np.vectorize(transform_y)(labels)

# 將結果轉換為 cupy 陣列
y = cp.array(transformed_labels_np)


In [47]:
X.shape

(10000, 1506)

In [50]:
type(y)

cupy.ndarray

In [48]:
y.shape

(10000,)

In [51]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Assuming X and y are your input and output arrays
# X.shape = (10000, 903), y.shape = (10000,)

def to_categorical_cupy(y, num_classes):
    # 確保 y 是 cupy 陣列
    y = cp.asarray(y)

    # 創建一個全零的矩陣
    one_hot = cp.zeros((y.size, num_classes), dtype=cp.float32)

    # 設置相應的 one-hot 值為 1
    one_hot[cp.arange(y.size), y] = 1
    return one_hot

# Convert y to one-hot encoded format for multi-class classification
y_one_hot = to_categorical_cupy(y, num_classes=5)
y_one_hot

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.1, random_state=42)


In [57]:
type(X_train)

cupy.ndarray

In [54]:
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    y_pred_classes = K.argmax(y_pred, axis=-1)
    y_true_classes = K.argmax(y_true, axis=-1)

    precision = tf.metrics.Precision()(y_true_classes, y_pred_classes)
    recall = tf.metrics.Recall()(y_true_classes, y_pred_classes)

    f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    return f1

In [34]:
import cupy as cp  # 確保安裝了 cupy
print("Num GPUs Available: ", cp.cuda.runtime.getDeviceCount())

Num GPUs Available:  1


In [60]:
X_train.shape

(9000, 1506)

In [63]:
model1 = Sequential()

# Input layer and first hidden layer (Dense)
model1.add(Dense(128, input_dim=1506, activation='relu'))  # Input dim matches the number of features in X

# Add additional hidden layers as needed
model1.add(Dense(64, activation='relu'))

# Output layer (for 5 classes)
model1.add(Dense(5, activation='softmax'))  # Softmax for multi-class classification


model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model1.fit(cp.asnumpy(X_train), cp.asnumpy(y_train), epochs=20, batch_size=32, validation_split=0.2)

loss, accuracy = model1.evaluate(cp.asnumpy(X_test), cp.asnumpy(y_test))
print(f"Test Accuracy: {accuracy:.2f}")


Epoch 1/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5322 - loss: 1.1644 - val_accuracy: 0.6044 - val_loss: 0.9342
Epoch 2/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6124 - loss: 0.9192 - val_accuracy: 0.6017 - val_loss: 0.9320
Epoch 3/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6316 - loss: 0.8641 - val_accuracy: 0.6717 - val_loss: 0.8537
Epoch 4/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6681 - loss: 0.7991 - val_accuracy: 0.6656 - val_loss: 0.8274
Epoch 5/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6739 - loss: 0.7869 - val_accuracy: 0.6828 - val_loss: 0.8028
Epoch 6/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6821 - loss: 0.7645 - val_accuracy: 0.6461 - val_loss: 0.8688
Epoch 7/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7065 - loss: 0.7267 - val_accuracy: 0.5633 - val_loss: 0.9752
Epoch 8/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7066 - loss: 0.7327 - val_accuracy: 0.

In [65]:
from sklearn.metrics import f1_score
import numpy as np

# Get predictions from the model (these will be probabilities)
y_pred_prob = model1.predict(cp.asnumpy(X_test))

# Convert probabilities to class predictions (e.g., the index of the max value)
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it back to class labels
y_test_class = np.argmax(cp.asnumpy(y_test), axis=1)

# Calculate the F1 score using sklearn
f1 = f1_score(y_test_class, y_pred, average='weighted')

# Print the F1 score
print(f"Test F1 Score: {f1:.2f}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Test F1 Score: 0.65
